In [1]:
### General ###

import numpy as np
import pandas as pd
import calendar

#### Visualization Tools ###

# Base of sns
import matplotlib.pyplot as plt

# Born on top of matplotlib, but more attractive
import seaborn as sns

# For interactive visualization, not good for too many datapoints
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go

In [2]:
df_viento = pd.read_csv('./meteochile/dataset/viento_dataset.csv', delimiter=';')
df_viento['momento'] = pd.to_datetime(df_viento['momento'])
df_viento

,codigoNacional,idEquipo,momento,ddInst,ffInst,dd02Minutos,ff02Minutos,dd10Minutos,ff10Minutos
0,360011,0,2019-02-05 18:27:00,218.0,9.7,218.0,4.2,212.0,7.8
1,360011,0,2019-02-05 18:28:00,208.0,8.0,212.0,4.5,213.0,7.7
2,360011,0,2019-02-05 18:29:00,230.0,6.2,204.0,3.8,214.0,7.2
3,360011,0,2019-02-05 18:30:00,213.0,6.8,218.0,3.2,219.0,7.2
4,360011,0,2019-02-05 18:31:00,232.0,9.3,223.0,4.0,221.0,7.5
...,...,...,...,...,...,...,...,...,...
2150608,360011,0,2023-07-31 23:56:00,28.0,4.7,34.0,2.3,44.0,3.2
2150609,360011,0,2023-07-31 23:57:00,28.0,4.9,28.0,2.5,41.0,3.4
2150610,360011,0,2023-07-31 23:58:00,34.0,5.2,28.0,2.5,39.0,3.6
2150611,360011,0,2023-07-31 23:59:00,32.0,5.4,30.0,2.6,37.0,3.8


In [3]:
df_llamados = pd.read_csv('./coelcha/llamados_clean.csv')
df_viento['momento'] = pd.to_datetime(df_viento['momento'])
df_llamados.head()

,Periodo,Número total de llamadas recibidas por mes (NLLR),Número total de llamadas atendidas en IVR por mes (NRIVR),Número total de llamadas desistidas en IVR por mes (NDIVR),Número total de llamadas atendidas por ejecutivo por mes (NREJC),Número total de llamadas cursadas por gestión de abandono por mes (NGA),Porcentaje de llamadas perdidas (LLP%),"Tasa de ocupación de las líneas, registradas con una frecuencia de una hora (OC)","Tiempo promedio de espera, considerando las llamadas atendidas por un ejecutivo (TPE)","Tiempo de conversación promedio, considerando las llamadas atendidas por un ejecutivo (TPC)"
0,2023-07-01,2907,700,0,2207,478,16.44,0,00:05:09,00:07:24
1,2023-06-01,3785,925,0,1972,554,14.63,0,00:04:51,00:07:09
2,2023-05-01,3342,816,0,1958,902,26.98,0,00:04:28,00:06:58
3,2023-04-01,4280,938,0,2285,1057,24.69,0,00:04:56,00:06:48
4,2023-03-01,3556,806,0,2163,587,16.50,0,00:04:33,00:06:14


In [4]:
from sklearn.preprocessing import MinMaxScaler

a = df_llamados[['Periodo', 'Número total de llamadas recibidas por mes (NLLR)']].copy()
b = df_viento[['momento', 'ffInst']].copy()
    
mindate = df_llamados['Periodo'].min()
maxdate = df_llamados['Periodo'].max()

b = b[b['momento'] >= mindate]
b = b[b['momento'] <= maxdate]

b = b.set_index('momento').resample('M').ffInst.mean().reset_index()
b['momento'] = b['momento'].apply(lambda x: x.replace(day=1))

a.rename(columns={'Periodo': 'momento'}, inplace=True)
a['momento'] = pd.to_datetime(a['momento'])

c = pd.merge(a, b, on='momento', how='inner')
c = c.set_index("momento")

scaler = MinMaxScaler()
c[['ffInst', 'Número total de llamadas recibidas por mes (NLLR)']] = scaler.fit_transform(c[['ffInst', 'Número total de llamadas recibidas por mes (NLLR)']])

# Plotting using plotly.express
fig = px.line(c, x=c.index, y=['ffInst', 'Número total de llamadas recibidas por mes (NLLR)'],
              title='Interacción Viento Llamados por mes',
              labels={'index': 'Index', 'value': 'Scaled Value'})

fig.show()